In [1]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

import pandas as pd
from pandas.io.json import json_normalize

In [6]:
authenticator = IAMAuthenticator('3HnVdWKI3sLAgms50soYv3zA4Kw-V2UIy6hM_aWkLpR1')

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze?version=2019-07-12')

text = """
..โปรเจกยักษ์ใหญ่ที่ ก่อสร้างโดย Singapore National Park
พื้นที่สีเขียวและเป็นแหล่งให้ความรู้ทางด้านพฤกษาศาสตร์แก่ประชาชนได้เป็นอย่างดี ..การแสดงแสงสีประกอบดนตรีที่ Super Trees ซึ่งสูงตั้งแต่ 25 -30 เมตร ปลูกต้นไม่จำพวกเถาและเฟิร์น ตอนกลางวันจะมีการเก็บพลังงานจากแสงอาทิตย์เพื่อนำมาเล่นเป็นแสงสียามค่ำคืน (พลาดไม่ชม ไม่ได้นะ..ขอบอกเลย)
ส่วนทางเดินชมสวนด้านบนเรียกว่า OCBC Skyway
ด้านบนหอกลางของต้นที่ใหญ่สุด จะเป็นร้านอาหาร ชมวิว 360 องศา
..ใครที่ไป Marina Bay Sands ใ้ห้เดินจากตึก Marina Bay Sands ให้เดินออกตรงทางเชื่อม ข้ามถนนมายัง Gardens by the Bay ได้ ประมาณ 5 นาที
..**แต่วันนั้น พวกเราออกจาก Marina Bay Sands แล้วแวะไปเช็คอิน Mandarin Oriental Singapore จากนั้นก็ไปกินมื้อกลางวันข้าวมันไก่ Boon Tong Kee ก่อน ก็
ต้องเริ่มเดินทางที่ สถานี MRT Novena มาลง MRT Marina Bay ก่อน (เพราะขี้เกียจต่อสายไปลงสถานี Bay front )
ข้อดีคือ เราเดินผ่านทางไปอุโมงค์รูปดอกไม้สีสันสวยงามจากสถานีใต้ดินนี้ไปยังสวน
....ราคาตั๋วผู้ใหญ่ เข้าชม Flower Dome + Cloud Forest อยู่ที่ 28 SGD ต่อคน
OCBC Skyway 5 SGd
....ส่วนราคาตั๋วเด็ก(3-12 yrs) Flower Dome + Cloud Forest อยู่ที่ 15 SGD ต่อคน OCBC Skyway 3 SGd
"""

response = natural_language_understanding.analyze(
    text=text,
    features=Features(entities=EntitiesOptions(emotion=True, sentiment=True, limit=100000), keywords=KeywordsOptions(emotion=True, sentiment=True, limit=100000))).get_result()

print(json.dumps(response, indent=2))

ERROR:root:unsupported text language: th
Traceback (most recent call last):
  File "/home/zyf0717/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 157, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: th, Code: 400 , X-global-transaction-id: c486dd422dbe05eac67fc2eb63df0f82


ApiException: Error: unsupported text language: th, Code: 400 , X-global-transaction-id: c486dd422dbe05eac67fc2eb63df0f82

In [3]:
keywords_col_names = ['WEBSITE_ID', 'REVIEW_ID', 'TEXT', 'RELEVANCE', 'COUNT',
                     'SENTIMENT_SCORE', 'SENTIMENT_LABEL', 'SADNESS', 'JOY',
                     'FEAR', 'DISGUST', 'ANGER', 'MIXED_SENTIMENT']

keywords_col_names_map = {
    'text': 'TEXT', 
    'relevance': 'RELEVANCE', 
    'count': 'COUNT', 
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER',
    'sentiment.mixed': 'MIXED_SENTIMENT'
}

keywords_col_names_df = pd.DataFrame(columns=keywords_col_names)

keywords_df = json_normalize(response['keywords']).rename(columns=keywords_col_names_map)
keywords_df.insert(0, 'REVIEW_ID', 123456)
keywords_df.insert(0, 'WEBSITE_ID', 1)
keywords_df = keywords_col_names_df.append(keywords_df, sort=False)
keywords_df


,WEBSITE_ID,REVIEW_ID,TEXT,RELEVANCE,COUNT,SENTIMENT_SCORE,SENTIMENT_LABEL,SADNESS,JOY,FEAR,DISGUST,ANGER,MIXED_SENTIMENT
0,1,123456,り体験,0.638770,3,0.554667,positive,NaN,NaN,NaN,NaN,NaN,1
1,1,123456,シンガポールの動物園の特徴,0.595085,1,0.406960,positive,NaN,NaN,NaN,NaN,NaN,1
2,1,123456,鳥たち,0.593169,1,-0.666857,negative,NaN,NaN,NaN,NaN,NaN,NaN
3,1,123456,シンガポール旅行〜,0.593019,1,0.000000,neutral,NaN,NaN,NaN,NaN,NaN,NaN
4,1,123456,シンガポールズー,0.571503,1,-0.666857,negative,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1,123456,動き,0.478585,2,-0.666857,negative,NaN,NaN,NaN,NaN,NaN,NaN
101,1,123456,タクシー,0.478222,2,0.673637,positive,NaN,NaN,NaN,NaN,NaN,NaN
102,1,123456,臨場感,0.478191,2,0.588403,positive,NaN,NaN,NaN,NaN,NaN,NaN
103,1,123456,参考,0.477610,2,0.000000,neutral,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
entities_col_names = ['WEBSITE_ID', 'REVIEW_ID', 'TYPE', 'TEXT', 'RELEVANCE',
                      'COUNT', 'CONFIDENCE', 'SENTIMENT_SCORE',
                      'SENTIMENT_LABEL', 'SADNESS', 'JOY', 'FEAR', 'DISGUST',
                      'ANGER', 'MIXED_SENTIMENT', 'DISAMBIGUATION_SUBTYPE', 'DISAMBIGUATION_NAME', 
                      'DISAMBIGUATION_RESOURCE']

entities_col_names_map = {
    'type': 'TYPE',
    'text': 'TEXT', 
    'relevance': 'RELEVANCE', 
    'count': 'COUNT',
    'confidence': 'CONFIDENCE',
    'sentiment.score': 'SENTIMENT_SCORE',
    'sentiment.label': 'SENTIMENT_LABEL',
    'emotion.sadness': 'SADNESS',
    'emotion.joy': 'JOY',
    'emotion.fear': 'FEAR',
    'emotion.disgust': 'DISGUST',
    'emotion.anger': 'ANGER',
    'sentiment.mixed': 'MIXED_SENTIMENT',
    'disambiguation.subtype': 'DISAMBIGUATION_SUBTYPE',
    'disambiguation.name': 'DISAMBIGUATION_NAME',
    'disambiguation.dbpedia_resource': 'DISAMBIGUATION_RESOURCE'
}

if 'entities' in response:
    entities_df = json_normalize(response['entities']).rename(columns=entities_col_names_map)
    entities_df.insert(0, 'REVIEW_ID', 1234567890)
    entities_df.insert(0, 'WEBSITE_ID', 1)
    entities_df.shape